In [1]:
import pandas as pd

In [2]:
#####

# README: This code turns x-y coordinate files into x-y-z files that can be used by Ovito

#####

# Change the zip code here to your coordinate file

zip_code = '15090'

s1 = 'Coords Folder/'

s2 = '_building_coords.txt'

file = open(s1 + zip_code + s2)

coords = pd.read_csv(file, sep = ',', names = ['x', 'y'])

file.close()

coords

,x,y
0,-80.064898,40.598676
1,-80.057230,40.605936
2,-80.053911,40.605764
3,-80.039885,40.639494
4,-80.042522,40.592529
...,...,...
9403,-80.074073,40.615258
9404,-80.078338,40.596541
9405,-80.040046,40.583516
9406,-80.057666,40.652279


In [3]:
# Create the normalized dataframe

normalized_data = pd.DataFrame()

for letter in ['x', 'y']:
    
    oldMax = coords[letter].max()
    
    oldMin = coords[letter].min()
    
    abs_max = max(abs(oldMax), abs(oldMin))
    
    normalized_data[letter] = coords[letter].divide(abs_max)
    
    mean = normalized_data[letter].mean()

    normalized_data[letter] = normalized_data[letter].subtract(mean)

    normalized_data[letter] = normalized_data[letter].multiply(10000)

normalized_data

,x,y
0,0.066413,-5.453327
1,1.023279,-3.668058
2,1.437517,-3.710406
3,3.187814,4.584133
4,2.858709,-6.964817
...,...,...
9403,-1.078517,-1.375794
9404,-1.610751,-5.978212
9405,3.167684,-9.181129
9406,0.968891,7.727891


In [4]:
normalized_data.insert(0, 'test', 'H')

normalized_data[''] = 0

normalized_data.loc[-1] = ['', '', '', '']

normalized_data.index = normalized_data.index + 1

normalized_data = normalized_data.sort_index()

r, c = normalized_data.shape

# Setting one of the 'z' values as 0.01 to prevent degeneracy (lol)

normalized_data.columns = ['1', '2', '3', '4']

normalized_data['4'][1] = 0.01

normalized_data.columns = [r - 1, '', '', '']

normalized_data

,9408,,,
0,,,,
1,H,0.0664128,-5.45333,0.01
2,H,1.02328,-3.66806,0
3,H,1.43752,-3.71041,0
4,H,3.18781,4.58413,0
...,...,...,...,...
9404,H,-1.07852,-1.37579,0
9405,H,-1.61075,-5.97821,0
9406,H,3.16768,-9.18113,0
9407,H,0.968891,7.72789,0


In [8]:
# Found here: https://stackoverflow.com/questions/41428539/data-frame-to-file-txt-python/41428596

normalized_data.to_csv(path_or_buf = s1 + zip_code + '_building_coords_reformatted.txt', index = False, header = True, sep = '\t', mode = 'a')